James Gardner, March 2022 

want to analyse science case/s for CE only:

CE-N 40km with CE-S 40km or 20km

if done, then look at CE-S with one ET detector

verify techniques by replicating *Borhanian and Sathya 2022*

In [ ]:
%load_ext autoreload
%autoreload 2

from networks import *
from detection_rates import *
from measurement_errors import *
from useful_functions import flatten_list

# suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

### Replicating Borhanian and Sathya 2022 injections and detection rates, then for CE only 

In [ ]:
# batch detection rate results for given science case and network set

# waveform, LAL list: https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__h.html
# --- BNS ---
# science_case = 'BNS'
# wf_model_name, wf_other_var_dic = 'lal_bns', dict(approximant='IMRPhenomD_NRTidalv2') # for tidal, see https://arxiv.org/abs/1905.06011
# to-do: missing dimensionless tidal parameters, calculate tidal parameters from sampled m1, m2 in injections.py? requires Love number and radii (i.e. choose an EoS)
# --- BBH ---
science_case = 'BBH'
wf_model_name, wf_other_var_dic = 'lal_bbh', dict(approximant='IMRPhenomHM')
# --- analytic waveforms ---
# wf_model_name, wf_other_var_dic = 'tf2', None # to-do: stop using this once tidal params found
# wf_model_name, wf_other_var_dic = 'tf2_tidal', None

# number of injections per redshift bin (6 bins)
num_injs = 10

# network_spec_list = BS2022_STANDARD_6['nets']
# network_spec_list = CE_ONLY['nets']
network_spec_list = CE_S_W_ET['nets']

for network_spec in tqdm(network_spec_list):
    detection_rate_for_network_and_waveform(network_spec, science_case, wf_model_name, wf_other_var_dic, num_injs, show_fig=False, print_progress=False, print_reach=False)

In [ ]:
# meta-batch to save all the required results (but not generate plots to save time) for the below cell
network_spec_list = NET_LIST

science_cases = ('BNS', 'BBH')
# --- numerical BNS as in B&S2021 ---
# wf_specifications = (('lal_bns', dict(approximant='IMRPhenomD_NRTidalv2')), ('lal_bbh', dict(approximant='IMRPhenomHM')))
# --- analytic BNS ---
wf_specifications = (('tf2_tidal', None), ('lal_bbh', dict(approximant='IMRPhenomHM')))

# number of injections per redshift bin (6 bins), tuple over science cases (more for symbolic wf.'s than numerical)
num_injs_list = (100, 10)

for network_spec in tqdm(network_spec_list):
    for j, science_case in enumerate(science_cases):
        wf_model_name, wf_other_var_dic = wf_specifications[j]        
        detection_rate_for_network_and_waveform(network_spec, science_case,
                                                wf_model_name, wf_other_var_dic, num_injs_list[j],
                                                generate_fig=True, show_fig=False,
                                                print_progress=False, print_reach=False)

### Collating different networks saved using the above method

In [ ]:
# batch of six collated plots: all three network sets and two science cases
# assumes results saved: above results generating cell must be run to save results for this cell

# ! remember to update plot label for each case else the plot will be overwritten
# how standard the "standard 6" are is unclear, seems like an invention of B&S2022
# CE only, with 2G+ as a reference
# CE South with one ET detector

# adding HLVKI+ reference to CE nets
network_dict_list = [NET_DICT_LIST[0],]
network_sets = [net_dict['nets'] for net_dict in network_dict_list]
#(BS2022_STANDARD_6['nets'],)
#                 CE_ONLY_C_and_S['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']],
#                 CE_ONLY_C_and_N['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']],
#                 CE_S_W_ET['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']])
network_labels = [net_dict['label'] for net_dict in network_dict_list]
#('standard_6', 'CE_only_C..S', 'CE_only_C..N', 'ET..CE_S')
science_cases = ('BNS', 'BBH')
specific_wfs = ('tf2_tidal', None)

for i, network_set in enumerate(network_sets):
    for j, science_case in enumerate(science_cases):
        plot_label = f'SCI-CASE_{science_case}_NET_{network_labels[i]}'
        specific_wf = specific_wfs[j]
        if specific_wf is not None:
            plot_label += f'_WF_{specific_wf}'
        compare_detection_rate_of_networks_from_saved_results(network_set, science_case, plot_label=plot_label, specific_wf=specific_wf, show_fig=False, data_path='data_20--200_injs/data_for_plots/') 

### Measurement errors

In [ ]:
# batching collation of standard sets assuming results saved
# compare plots to B&S2022 Figs 3 and 4

network_dict_list = [NET_DICT_LIST[0],]
network_sets = [net_dict['nets'] for net_dict in network_dict_list]
network_labels = [net_dict['label'] for net_dict in network_dict_list]
# network_sets = (BS2022_STANDARD_6['nets'],
#                 CE_ONLY_C_and_S['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']],
#                 CE_ONLY_C_and_N['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']],
#                 CE_S_W_ET['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']])
# network_labels = ('standard_6', 'CE_only_C..S', 'CE_only_C..N', 'ET_E..CE_S')
science_cases = ('BNS', 'BBH')
specific_wfs = ('tf2_tidal', None)

for i, network_set in enumerate(tqdm(network_sets)):
    for j, science_case in enumerate(science_cases):
        plot_label = f'SCI-CASE_{science_case}_NETS_{network_labels[i]}'
        plot_title = f'Science case: {science_case}, network set: {network_labels[i]}'
        specific_wf = specific_wfs[j]
        if specific_wf is not None:
            plot_label += f'_WF_{specific_wf}'
            plot_title += f', WF: {specific_wf}'
        collate_measurement_errs_CDFs_of_networks(network_set, science_case, specific_wf=specific_wf, plot_label=plot_label, plot_title=plot_title, full_legend=False, print_progress=False, show_fig=False, normalise_count=True, xlim_list=None, threshold_by_SNR=True, CDFmin=1e-4, data_path='data_20--200_injs/data_for_plots/')
#         # for reference, without SNR thresholding
#         collate_measurement_errs_CDFs_of_networks(network_set, science_case, specific_wf=specific_wf, plot_label=plot_label+'_SNR-THR_False', plot_title=plot_title, full_legend=False, print_progress=False, show_fig=False, normalise_count=True, xlim_list=None, threshold_by_SNR=False)

In [ ]:
# for direct comparison, use B&S2022's xlim_list 
collate_measurement_errs_CDFs_of_networks(BS2022_SIX['nets'], 'BNS', specific_wf='tf2_tidal', plot_label='SCI-CASE_BNS_NETS_standard_6_WF_tf2_tidal_XLIMS_preset', plot_title='Science case: BNS, network set: standard 6, WF: tf2_tidal, XLIMS: B&S2022 preset', full_legend=False, print_progress=False, show_fig=True, normalise_count=True, xlim_list='B&S2022', threshold_by_SNR=False, CDFmin=1e-4, data_path='data_20--200_injs/data_for_plots/')

In [ ]:
# B&S2022 plots cumulative density functions (CDF) versus redshift to show distribution of measurement errors (why not just use PDF, i.e. normalised histograms, like B2021?)
# for non-SNR quantities only use results above an SNR threshold (e.g. 10), alternatively, for all non--sky area quantities only use results below a 90% credible sky area threshold (e.g. 4.4e-2 sqrDeg, where the full Moon is ~20.0e-2 sqrDeg given a radius of 0.5 deg) --> don't have enough resolution to attempt sky-area thresholding

# ax.hist(data, histtype='step', facecolor='b', label='tf2_tidal', bins=np.geomspace(data.min(), data.max(), 50))

###  To-do list

In [ ]:
# to-do: stop CE alone being ill-conditioned, seems to work for BBH numeric wf --> get latest patch from Borhanian
# to-do: fix BNS numerical waveform, current error "numpy.linalg.LinAlgError: Array must not contain infs or NaNs" with HLVKI+ --> see above

In [ ]:
# to-do: change sigmoid fits to global optimisation to avoid needing the initial guesses in B&S2022, this is not possible with scipy, need to write my own or source a global curve fitting (apparantly a non-trivial problem)

In [ ]:
# to-do: refactor two plotting functions 
# to-do: tidy up refactoring, reduce total number of arguments with unpacking?

In [ ]:
"""In fact, we see that the three generations (A+, Voyager, and NG) are qualitatively different
with respect to every metric used in this study."""